# Joining Tables

In [1]:
# setting up modules

import pandas as pd
import numpy as np
import os
import janitor

In [2]:
# loading in data

path = r'/Users/johntam/cf_data_analysis/tasks/A4/code/data'
orders_prior = pd.read_csv(os.path.join(path,'order_products__prior.csv'))
products = pd.read_csv(os.path.join(path,"products.csv"))


## data checks for products table

In [3]:
## checking for nan's and removing them
products.isnull().sum()

clean_products = products[products['product_id'].isnull() == False]

In [4]:
## looking for dups
clean_products[clean_products.duplicated()]

cleaner_products = clean_products.drop_duplicates()

In [5]:
## seeing if product id is still duplicated 

cleaner_products[cleaner_products.product_id.duplicated()]

,product_id,product_name,aisle_id,department_id,prices
6800,6800,Sprouted Quinoa Flakes Baby Cereal,92,18,14.0
26521,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9


In [6]:
# exporting to csv
path = r'/Users/johntam/cf_data_analysis/tasks/A4/code/clean_data'
cleaner_products.to_csv(os.path.join(path, 'products_wrangled.csv'))

## data checks for order tables

In [7]:
path = r'/Users/johntam/cf_data_analysis/tasks/A4/code/clean_data'
orders = pd.read_csv(os.path.join(path,'orders_wrangled.csv'))

In [31]:
orders_prior.shape
orders_prior.describe()

,order_id,product_id,add_to_cart_order,reordered
count,3.243449e+07,3.243449e+07,3.243449e+07,3.243449e+07
mean,1.710749e+06,2.557634e+04,8.351076e+00,5.896975e-01
std,9.873007e+05,1.409669e+04,7.126671e+00,4.918886e-01
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,8.559430e+05,1.353000e+04,3.000000e+00,0.000000e+00
50%,1.711048e+06,2.525600e+04,6.000000e+00,1.000000e+00
75%,2.565514e+06,3.793500e+04,1.100000e+01,1.000000e+00
max,3.421083e+06,4.968800e+04,1.450000e+02,1.000000e+00


In [28]:
# seeing if there are any duplicated in order id
orders.order_id.duplicated().sum()

0

In [32]:
orders.describe()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


## merging and exporting data

In [8]:
# merging the two dataframes

orders_large = orders.merge(orders_prior, on = 'order_id', indicator = True)

In [9]:
orders_large._merge.value_counts()

both          32434489
left_only            0
right_only           0
Name: _merge, dtype: int64

In [10]:
# exporting data
path = r'/Users/johntam/cf_data_analysis/tasks/A4/code/clean_data'
orders_large.to_pickle(os.path.join(path, 'orders_products_combined.pkl'))

In [11]:
# reading in data
orders_full = pd.read_pickle(os.path.join(path,'orders_products_combined.pkl')).drop(columns = ['_merge'])

In [9]:
products = pd.read_csv(os.path.join(path, 'products_wrangled.csv'))

In [12]:
# there are two products with the same product id 32404859

products[products.product_id.isin([6799,6800,26519,26520,26521])]

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices
6798,6799,6800,Revive Zero Vitamin Water,64,7,6.4
6799,6800,6800,Sprouted Quinoa Flakes Baby Cereal,92,18,14.0
26518,26520,26520,Clinical Advanced Solid Ultimate Fresh Anti-Pe...,80,11,10.6
26519,26521,26520,Cheese Shredded Sharp Cheddar Reduced Fat 2%,21,16,2.9
26520,26522,26521,Organic Grass-Fed Classic Mild Cheddar Macaron...,4,9,9.0


In [69]:
#pd.merge(orders_full,products, how='left', on = 'product_id', indicator = True).value_counts('_merge')

_merge
both          32433030
left_only         2029
right_only           0
dtype: int64

In [70]:
#pd.merge(products,orders_full, how='left', on = 'product_id', indicator = True).value_counts('_merge')

_merge
both          32433030
left_only           11
right_only           0
dtype: int64

In [12]:
orders_products_merged = orders_full.merge(products, how='inner', on='product_id')

In [13]:
# exporting the data
orders_products_merged.to_pickle(os.path.join(path, 'orders_products_merged.pkl'))